In [30]:
# import dependencies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:

# Data from: https://www.kaggle.com/datasets/ermismbatuhan/digital-marketing-ecommerce-customer-behavior
# Digital Marketing | E-Commerce | Customer Behavior
# Create a dataframe first
# Please note the  csv file note seperated by commas, but semi colons hence adding the "sep"
df = pd.read_csv("data1.csv", sep = ";")

In [5]:
df.head()

,account length,location code,user id,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,churn
0,128,415,3824657,no,yes,25,265,45,17,110,197,87,"244,7",91,"11,01",10,3,"2,7",1,0
1,107,415,3717191,no,yes,26,162,27,17,123,196,103,"254,4",103,"11,45","13,7",3,"3,7",1,0
2,137,415,3581921,no,no,0,243,41,10,114,121,110,"162,6",104,"7,32","12,2",5,"3,29",0,0
3,84,408,3759999,yes,no,0,299,51,5,71,62,88,"196,9",89,"8,86","6,6",7,"1,78",2,0
4,75,415,3306626,yes,no,0,167,28,13,113,148,122,"186,9",121,"8,41","10,1",3,"2,73",3,0


In [7]:
# lets now investigate the data frame / the shape of our data
df.shape

(3333, 20)

In [8]:
# lets get more infromation
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   account length                       3333 non-null   int64 
 1   location code                        3333 non-null   int64 
 2   user id                              3333 non-null   int64 
 3   credit card info save                3333 non-null   object
 4   push status                          3333 non-null   object
 5   add to wishlist                      3333 non-null   int64 
 6   desktop sessions                     3333 non-null   int64 
 7   app sessions                         3333 non-null   int64 
 8   desktop transactions                 3333 non-null   int64 
 9   total product detail views           3333 non-null   int64 
 10  session duration                     3333 non-null   int64 
 11  promotion clicks                     3333 n

In [9]:
# need to change data types, no null values
df["location code"].unique()

array([415, 408, 510])

In [10]:
df["location code"] = df["location code"].astype(str)

In [13]:
# change all categorical into 0 and i (intergers)
df["credit card info save"].unique()

array(['no', 'yes'], dtype=object)

In [14]:
df["credit card info save"] = df["credit card info save"].replace({"yes": 1, "no":0})
df["push status"] = df["push status"].replace({"yes": 1, "no":0})

In [15]:
# Change all floats into intergers
df["avg order value"] = df["avg order value"].str.replace(",",".").astype(float)
df["discount rate per visited products"] = df["discount rate per visited products"].str.replace(",",".").astype(float)
df["product detail view per app session"] = df["product detail view per app session"].str.replace(",",".").astype(float)
df["add to cart per session"] = df["add to cart per session"].str.replace(",",".").astype(float)

In [16]:
# lets hand categorial value for loaction code
df = pd.get_dummies(df, columns = ["location code"])

In [17]:
# for classification problem we don't need user ID, for now
df = df.drop("user id", axis = 1 )

In [18]:
# lets check dataframe after preprocessing
df.head()

,account length,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,...,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,churn,location code_408,location code_415,location code_510
0,128,0,1,25,265,45,17,110,197,87,...,91,11.01,10.0,3,2.70,1,0,0,1,0
1,107,0,1,26,162,27,17,123,196,103,...,103,11.45,13.7,3,3.70,1,0,0,1,0
2,137,0,0,0,243,41,10,114,121,110,...,104,7.32,12.2,5,3.29,0,0,0,1,0
3,84,1,0,0,299,51,5,71,62,88,...,89,8.86,6.6,7,1.78,2,0,1,0,0
4,75,1,0,0,167,28,13,113,148,122,...,121,8.41,10.1,3,2.73,3,0,0,1,0


In [19]:
# we use Normaliser which we imported from sklearn preprocessing to scale all data to be uniform as that will affect model
# use all columns
df.columns


Index(['account length', 'credit card info save', 'push status',
       'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls', 'churn',
       'location code_408', 'location code_415', 'location code_510'],
      dtype='object')

In [22]:
#  will remove the 0,1
cols_to_scale= ['account length',
       'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls']
scaler = Normalizer()
scaled_data = scaler.fit_transform(df[cols_to_scale])
scaled_df = pd.DataFrame(scaled_data, index = df.index, columns = cols_to_scale)

In [23]:
scaled_df.head()

,account length,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls
0,0.275142,0.053739,0.569631,0.096730,0.036542,0.236451,0.423461,0.187011,0.525995,0.195609,0.023667,0.021496,0.006449,0.005804,0.002150
1,0.252755,0.061417,0.382676,0.063779,0.040157,0.290551,0.462991,0.243307,0.600944,0.243307,0.027047,0.032362,0.007087,0.008740,0.002362
2,0.345945,0.000000,0.613611,0.103531,0.025251,0.287867,0.305543,0.277766,0.410589,0.262615,0.018484,0.030807,0.012626,0.008308,0.000000
3,0.208327,0.000000,0.741543,0.126484,0.012400,0.176086,0.153765,0.218247,0.488327,0.220727,0.021973,0.016369,0.017361,0.004415,0.004960
4,0.205041,0.000000,0.456559,0.076549,0.035541,0.308929,0.404615,0.333534,0.510963,0.330800,0.022992,0.027612,0.008202,0.007464,0.008202


In [24]:
# merged scaled_df with orginal dataframe
# drop colums to avoid duplication
df = df.drop(cols_to_scale, axis = 1)
df = pd.merge(df, scaled_df, left_index=True, right_index=True)

In [25]:
# divide into trained data and test data
# train test data
X = df.drop("churn", axis=1)
y = df["churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2233, 20) (1100, 20) (2233,) (1100,)


In [28]:
# time to build the model
# Going to to XGB, classification model
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [29]:
# predict our y from our x test dat frame using same model
preds = xgb_cl.predict(X_test)

In [31]:
acc = accuracy_score(y_test, preds)

In [32]:
print("Model Accuracy for Test Data Set:", acc)

Model Accuracy for Test Data Set: 0.92
